In [1]:
import math
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\dandr\anaconda3\Lib\site-packages\ortools\.libs\ortools.dll...


In [ ]:
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

denver_airport = ("Denver Airport", (39.8571391027151, -104.67682422525337))
resorts = [
    denver_airport,
    ("Vail",        (39.60634275473704, -106.35502534758554)),
    ("Beaver Creek",(39.601840601122596, -106.53163266107806)),
    ("Breckenridge",(39.48111351412133, -106.07307234912773)),
    ("Keystone",    (39.58186424927065, -105.94362588806374)),
    ("A-Basin",     (39.634289749362566, -105.8714993899124)),
    ("Eldora",      (39.937376571357305, -105.58271078989766)),
    ("Canyons",     (40.685834569379516, -111.55630231684529)),
    ("Heavenly",    (38.9288904430737, -119.90519040528994)),
    ("Northstar",   (39.26481569676928, -120.13316534575092)),
    ("Kirkwood",    (38.68503611762454, -120.06520941879404)),
    ("Afton Alps",  (44.85782048122588, -92.78779547430095)),
    ("Mt. Brighton",(42.5410196899345, -83.81160753394613)),
    ("Verbier",     (46.09708737973264,   7.227272162830475)),
    ("Arlberg",     (47.129520437009155, 10.263859008628906)),
    ("The 3 Valleys",(45.34154189826385,  6.586552723873957))
]

num_locations = len(resorts)
distance_matrix = [
    [
        0 if i == j else int(haversine(resorts[i][1], resorts[j][1]))
        for j in range(num_locations)
    ]
    for i in range(num_locations)
]

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

In [ ]:
start_index   = 0  # DEN
best_route    = None
best_distance = float('inf')
best_end      = None

# loop over every possible end node to find the shortest path
for end_index in range(num_locations):
    if end_index == start_index:
        continue

    # manager & model for this (start, end) pair
    manager = pywrapcp.RoutingIndexManager(
        num_locations,
        1,                # one vehicle
        [start_index],    # start node list
        [end_index]       # end node list
    )
    routing = pywrapcp.RoutingModel(manager)

    # register the distance callback
    def make_callback(dist_mat, mgr):
        def distance_callback(from_idx, to_idx):
            from_node = mgr.IndexToNode(from_idx)
            to_node   = mgr.IndexToNode(to_idx)
            return dist_mat[from_node][to_node]
        return distance_callback

    transit_cb = routing.RegisterTransitCallback(
        make_callback(distance_matrix, manager)
    )
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

    solution = routing.SolveWithParameters(search_parameters)
    if solution is None:
        continue

# Finding the best route

In [ ]:
index      = routing.Start(0)
route      = []
route_dist = 0
while not routing.IsEnd(index):
    node = manager.IndexToNode(index)
    route.append(resorts[node][0])
    prev_index = index
    index = solution.Value(routing.NextVar(index))
    route_dist += routing.GetArcCostForVehicle(prev_index, index, 0)
route.append(resorts[manager.IndexToNode(index)][0])


if route_dist < best_distance:
    best_distance = route_dist
    best_route    = route
    best_end      = end_index

print(f"Best endpoint: {resorts[best_end][0]}")
print("Open-TSP optimal route:")
for stop in best_route:
    print("  ", stop)
print(f"Total distance (km): {best_distance}")

Best endpoint: The 3 Valleys
Open-TSP optimal route:
   Denver Airport
   Eldora
   A-Basin
   Keystone
   Breckenridge
   Vail
   Beaver Creek
   Kirkwood
   Heavenly
   Northstar
   Canyons
   Afton Alps
   Mt. Brighton
   Arlberg
   Verbier
   The 3 Valleys
Total distance (km): 11850


# Cost Calculation

In [ ]:
def km_to_miles(km):
    return km / 1.60934

drive_cost_per_mile = 0.58
flight_variable_cost = 0.14
flight_base_fee = 50
threshold_miles = 400
lodging_per_stop = 120

total_cost = 0
for i in range(1, len(best_route)):
    name1 = best_route[i - 1]
    name2 = best_route[i]

    coord1 = dict(resorts)[name1]
    coord2 = dict(resorts)[name2]

    dist_km = haversine(coord1, coord2)
    dist_miles = km_to_miles(dist_km)

    if dist_miles > threshold_miles:
        transport_cost = flight_base_fee + dist_miles * flight_variable_cost
    else:
        transport_cost = dist_miles * drive_cost_per_mile

    lodging_cost = lodging_per_stop

    total_cost += transport_cost + lodging_cost

# Remove starting point lodging cost
total_cost -= lodging_per_stop

print(f"\nTotal Open-TSP cost: ${total_cost:,.2f}")


Total Open-TSP cost: $3,130.04
